In [ ]:
# %pip install -qU langchain-openai

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# %pip install langchain-chroma

In [ ]:
# %pip install langchain

In [ ]:
# %pip install langchain langchain_community langchain_chroma

In [ ]:
# %pip install langchainhub

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://sih.gov.in/sih2024PS",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("prob-statement-panel"))),
    verify_ssl=False,
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is the problem statement of SIH 2024?")

## Output of above query is:

'The problem statement for SIH 2024 focuses on "Student Innovation" with various themes. One key statement is to develop ideas that showcase the rich cultural heritage and traditions of India. This encompasses both hardware and software solutions under the categories of Heritage & Culture and Fitness & Sports.'


In [ ]:
rag_chain.invoke("I am a full stack developer. What problem statements can I work on?")

## Output of the above query is:

'As a full stack developer, you can work on creating an interactive job and internship platform for the Technical Education Department in Rajasthan, addressing the gap in job access and support for graduates. Another option is to develop a freelancing platform that connects freelancers with short-term job opportunities and includes project management tools. Additionally, you could build a mobile application to provide recreational suitability information for beach locations across India, focusing on innovative solutions.'

In [ ]:
# cleanup
vectorstore.delete_collection()